In [8]:
!pip install pandas
!pip install eventregistry

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'eventregistry' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'eventregistry'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for eventregistry: filename=eventregistry-9.1-py3-none-any.whl size=67624 sha256=8ac14cfe905d36870b60a65ddab87e684e1bc840d9f68790bf354a4b2ae3ee59
  Stored in directory: /root/.cache/pip/wheels/ef/b4/df/d901df881884202dc1fcb1a2ce42852f26ecf6fd1200ebf2b8
Successfully built eventregistry


In [9]:
import pandas as pd
from eventregistry import *
from datetime import date, timedelta


er = EventRegistry(apiKey="ced441ab-7fa4-4e20-b4af-8bd02fbb5431")

italian_source_uris = [
    'ansa.it', 'corriere.it', 'repubblica.it', 'lastampa.it', 'ilsole24ore.com',
    'adnkronos.com', 'ilfattoquotidiano.it', 'ilmessaggero.it', 'ilgiornale.it',
    'avvenire.it', 'internazionale.it', 'ilmattino.it', 'ilgazzettino.it',
    'unionesarda.it', 'ilsecoloxix.it', 'gds.it', 'quotidiano.net', 
    'liberoquotidiano.it', 'huffingtonpost.it', 'fanpage.it'
]


end_date = date.today()
start_date = end_date - timedelta(days=30)

all_articles = []
max_articles_to_fetch = 10000

print(f"Inizio raccolta per un massimo di {max_articles_to_fetch} articoli italiani da {start_date} a {end_date}.")
print(f"Numero di fonti utilizzate: {len(italian_source_uris)}")

# Definisci quali informazioni vuoi, usando il parametro corretto 'returnInfo'
requested_info = RequestArticlesInfo(
    returnInfo=ReturnInfo(
        articleInfo=ArticleInfoFlags(
            body=True,
            title=True,
            url=True,
            source=SourceInfoFlags(title=True),
            dateTimePub=True
        )
    )
)

q = QueryArticlesIter(
    sourceUri=QueryItems.OR(italian_source_uris),
    dateStart=start_date,
    dateEnd=end_date,
    lang="ita",
    requestedResult=requested_info
)

print(f"query: La libreria scaricherà gli articoli in blocchi.")
try:
    for article in q.execQuery(er, sortBy="date", max_items=max_articles_to_fetch):
        all_articles.append({
            'source': article.get('source', {}).get('title', 'N/A'),
            'title': article.get('title'),
            'description': article.get('body', '')[:300] + '...' if article.get('body') else '',
            'content': article.get('body'),
            'published_at': article.get('dateTimePub'),
            'url': article.get('url')
        })

        if len(all_articles) > 0 and len(all_articles) % 250 == 0:
            print(f"  -> Raccolti {len(all_articles)} articoli...")

except Exception as e:
    print(f"\nERRORE durante il download degli articoli: {e}")
    print("Potrebbe essere stato raggiunto il limite del piano gratuito o un altro problema con la richiesta.")

df = pd.DataFrame(all_articles)
df.to_csv('articoli_italia_ultimi_30_giorni.csv', index=False)
print(f"\nRaccolta completata! Totale articoli salvati: {len(df)}")

Inizio raccolta per un massimo di 10000 articoli italiani da 2025-08-03 a 2025-09-02.
Numero di fonti utilizzate: 20
Eseguo la query... La libreria scaricherà gli articoli in blocchi.
  -> Raccolti 250 articoli...
  -> Raccolti 500 articoli...
  -> Raccolti 750 articoli...
  -> Raccolti 1000 articoli...
  -> Raccolti 1250 articoli...
  -> Raccolti 1500 articoli...
  -> Raccolti 1750 articoli...
  -> Raccolti 2000 articoli...
  -> Raccolti 2250 articoli...
  -> Raccolti 2500 articoli...
  -> Raccolti 2750 articoli...
  -> Raccolti 3000 articoli...
  -> Raccolti 3250 articoli...
  -> Raccolti 3500 articoli...
  -> Raccolti 3750 articoli...
  -> Raccolti 4000 articoli...
  -> Raccolti 4250 articoli...
  -> Raccolti 4500 articoli...
  -> Raccolti 4750 articoli...
  -> Raccolti 5000 articoli...
  -> Raccolti 5250 articoli...
  -> Raccolti 5500 articoli...
  -> Raccolti 5750 articoli...
  -> Raccolti 6000 articoli...
  -> Raccolti 6250 articoli...
  -> Raccolti 6500 articoli...
  -> Raccolti